# Model Persistence
Save and load pre-trained models

This example demonstrates how to save trained deep-river models to disk and load them later for continued training or inference.

In [ ]:
import torch
import torch.nn as nn
from river import datasets, metrics
import tempfile
import os
from deep_river.classification import Classifier
from deep_river import base

## Create and Train a Classification Model

We will create a simple neural network for phishing detection and train it on some data.

In [ ]:
dataset = datasets.Phishing()
metric = metrics.Accuracy()

class SimpleNet(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.dense0 = nn.Linear(n_features, 5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X

model = Classifier(
    module=SimpleNet(n_features=9),
    loss_fn="binary_cross_entropy",
    optimizer_fn="adam"
)
model

Train the model on 1000 samples from the phishing dataset:

In [ ]:
# Train the model
for x, y in dataset.take(1000):
    y_pred = model.predict_one(x)
    if y_pred is not None:
        metric.update(y, y_pred)
    model.learn_one(x, y)
    
print(f"Accuracy: {metric.get():.3f}")

## Save the Trained Model

Now we will save the trained model to disk using the built-in save functionality:

In [ ]:
# Save the model
temp_dir = tempfile.mkdtemp()
model_path = os.path.join(temp_dir, "classifier.pkl")

model.save(model_path)
print(f"Model saved to: {model_path}")

## Load the Model and Test

Load the saved model and verify it works correctly:

In [ ]:
# Load the model
loaded_model = base.load_model(model_path)
print(f"Model loaded: {type(loaded_model).__name__}")

# Test prediction
test_sample = {"empty_server_form_handler": 0.0, "popup_window": 0.0, "https": 0.0, 
               "request_from_other_domain": 0.0, "anchor_from_other_domain": 0.0, 
               "is_popular": 0.5, "contains_submit_button": 1.0, "iframe": 0.0, 
               "is_from_cache": 0.0}

prediction = loaded_model.predict_one(test_sample)
print(f"Prediction: {prediction}")

## Cleanup

Clean up temporary files:

In [ ]:
# Clean up
import shutil
shutil.rmtree(temp_dir)
print("Cleanup completed")